In [1]:
import wget
import os
import time

from datetime import date

import pandas as pd
from bizdays import Calendar

# PARTE 1

Baixar o arquivo BVBG.028 no site da B3 

url: https://www.b3.com.br/pt_br/market-data-e-indices/servicos-de-dados/market-data/historico/boletins-diarios/pesquisa-por-pregao/pesquisa-por-pregao/

Com este arquivo, poderemos extrair os códigos de negociação (Tickers) para posteriormente buscarmos o preço
históricos destes ativos na B3.

In [2]:
def get_bvbg28(file_date):
    
    '''
    ** PASSO 1: Baixar o arquivo BVBG.028 no site da B3 **
    Cadastro de instrumentos - BVBG.028.01 Instruments File
        - Este arquivo contém as características dos instrumentos negociáveis e dos 
          instrumentos aceitos em garantia que são de conhecimento público.
    '''
    
    file_path = os.path.join(os.getcwd(), 'download')      # O caminho de onde o arquivo será baixado
    check_existence = os.path.exists(file_path)            # Faço uma checagem inicial, para verificar se arquivo já existe
    
    if check_existence == True:
        raise ValueError("Delete o arquivo 'download' do seu atual diretório, antes de rodar este código.")
    
    # Garantimos que não existe nenhum arquivo chamado "Download" no diretório alvo
    else:
        pre_url='https://www.b3.com.br/pesquisapregao/download?filelist='   # Criamos uma pré-url que será add o nome do arquivo
        full_url = f'{pre_url}{file_date}.zip'                              # O Endereço completo da requisição que eu vou fazer
        wget.download(full_url)                                             # Efetivamente baixar o arquivo BVBG28
        time.sleep(20)

        check_download = os.path.exists(file_path)                          # Faço uma checagem Final, para verificar se arquivo já existe

        if check_download == True: 
            confirmation = print('Arquivo baixado com sucesso')
            
        else:
            confirmation = print('Ocorreu um problema com o Download do arquivo')

    return confirmation

In [3]:
def show_yday(calendar_file, user_offset):
    f = open(calendar_file, 'r').read().split('\n')
    holidays = [d.date() for d in pd.to_datetime(f)]

    cal = Calendar(holidays, ['Sunday', 'Saturday'])

    today = date.today()
    ontem = cal.offset(today, user_offset)

    return ontem

In [4]:
'''
ATENÇÃO: O Nome do arquivo muda todo dia e é composto por:
    1) Inicial "IN"
    2) Data do arquivo, no formato "YYMMDD"
'''

anbima_file = os.path.join(os.getcwd(), 'ANBIMA.txt')

my_day_parameter = show_yday(calendar_file=anbima_file, user_offset=-1)
ajusted_day = my_day_parameter.strftime("%Y%m%d")[2:]

sufixo = f'IN{ajusted_day}'

download_bvbg_zip = get_bvbg28(file_date=sufixo)

Arquivo baixado com sucesso


# PARTE 2

Retirar as informações dos CÓDIGOS DE NEGOCIAÇÃO (Ticker) dos arquivos e organiza-los em um DataFrame

In [5]:
import zipfile
import xml.etree.ElementTree as ET
import warnings

In [6]:
def unextracting(zip_file, dir_target):
    try:
        with zipfile.ZipFile(zip_file) as z:
            z.extractall(dir_target)
            print("Extracted all")
    except:
        print("Invalid file")

In [7]:
# Porque o zip esta dentro de outro zip, teremos que realizar a extração 2 vezes

# Extração número_01
try_one = unextracting(zip_file="download", dir_target=os.getcwd())

# Criar um diretório chamado extracted_files
os.mkdir('extracted_files')  
file_destination = os.path.join(os.getcwd(), 'extracted_files')

# Extração número_02
try_two = unextracting(zip_file=f'{sufixo}.zip', dir_target=file_destination)

# Remover os arquivos .zip que ficaram de bobeira
os.remove("download")
os.remove(f'{sufixo}.zip')

# Mudança de visão de diretório
os.chdir(file_destination)

Extracted all
Extracted all


-----------------

# LAMBDAS

Lambdas são "funções anonimas" que podem ser feitas de forma rápida. Este tipo de função se comporta exatamente como uma UDF (User Defined Function) - as mesmas que temos utilizados até então - com a diferença que deve conter uma expressão simples.

<strong>Funções do tipo Lambda podem retornar objetos</strong>


## Criando uma função Lambda
lambda argument(s): expression

Por exemplo:

In [8]:
# Função Lambda
my_lambda_example = lambda x: x + 2
exemplo_01 = my_lambda_example(5)
print(f'Resultado do exemplo 01, usando um função LAMBDA: {exemplo_01}')


# Função UDF
def exemplo_02(x):
    x = x + 2
    return x

exemplo_02 = exemplo_02(5)
print(f'Resultado do exemplo 02, usando um UDF tradicional: {exemplo_02}')

Resultado do exemplo 01, usando um função LAMBDA: 7
Resultado do exemplo 02, usando um UDF tradicional: 7


-----------------

In [9]:
fun = lambda x : os.path.isfile(os.path.join(file_destination,x))  # Extrair uma lista com todos os arquivos dentro de um diretório
print(type(fun))                                                   # Demonstrando que o retorno é um objeto

files_list = filter(fun, os.listdir(file_destination))             # Comparando o objeto com a lista de arquivos no diretório

# Criando a lista com o nome dos aquivos juntamento com o tamanho
size_of_file = [
        (f,os.stat(os.path.join(file_destination, f)).st_size)
        for f in files_list
]

# Criando uma função para na apresentação dos arquivos conforme seu tamanho
fun = lambda x : x[1]

# Inserindo o nome arquivos em 1 lista e o tamanho em outra, de forma ordenada
list_files =[]                                    # list_files = Contem o nome dos arquivos no diretório
list_size = []                                    # list_size = Contem o tamanho dos arquivos no diretório
for f,s in sorted(size_of_file):
    file_name = f
    list_files.append(file_name)
    
    size = round(s/(1024*1024),3)
    list_size.append(size)
    
# Verificando a posição (index) do maior maior arquivo na lista "list_size".
max_num = 0
for i in list_size:
    if i > max_num:
        max_num = i    
    
# Verificando o nome do arquivo de mesma posição (index) para o maior tamanho encontrado na etapa anterior
reference = list_size.index(max_num)
largest_file = list_files[reference]
print(f'O maior arquivo desta pasta é o {largest_file}')

<class 'function'>
O maior arquivo desta pasta é o BVBG.028.02_BV000327202202140327103407280418033.xml


In [10]:
warnings.filterwarnings("ignore", category=DeprecationWarning) # Ignorar avisos (tela vermelha chata)

tree = ET.parse(largest_file)                     # Criando um elemento contendo a arvóre do XML                    
root = tree.getroot()                             # Raiz do elemento criado

my_drive = [elem.tag for elem in root.iter()]     # Caso você queira verificar todos os elementos da árvore
for i in my_drive[0:20]:
    print(i)

{urn:bvmf.052.01.xsd}Document
{urn:bvmf.052.01.xsd}BizFileHdr
{urn:bvmf.052.01.xsd}Xchg
{urn:bvmf.052.01.xsd}BizGrpDesc
{urn:bvmf.052.01.xsd}Fr
{urn:bvmf.052.01.xsd}OrgId
{urn:bvmf.052.01.xsd}Id
{urn:bvmf.052.01.xsd}OrgId
{urn:bvmf.052.01.xsd}Othr
{urn:bvmf.052.01.xsd}Id
{urn:bvmf.052.01.xsd}Issr
{urn:bvmf.052.01.xsd}SchmeNm
{urn:bvmf.052.01.xsd}Prtry
{urn:bvmf.052.01.xsd}To
{urn:bvmf.052.01.xsd}OrgId
{urn:bvmf.052.01.xsd}Id
{urn:bvmf.052.01.xsd}OrgId
{urn:bvmf.052.01.xsd}Othr
{urn:bvmf.052.01.xsd}Id
{urn:bvmf.052.01.xsd}Issr


In [11]:
extracted_information ={'TICKER':[]}
for child in root:
    for layer2 in child.getchildren():
        for layer3 in layer2.getchildren():
            for layer4 in layer3.getchildren():
                for layer5 in layer4.getchildren():
                    for layer6 in layer5.getchildren():
                        for layer7 in layer6.getchildren():
                            for layer8 in layer7.getchildren():
                                ticker = [j.text for j in layer8.iter("{urn:bvmf.100.02.xsd}TckrSymb")]
                                for j in ticker:
                                    if len(j) == 6:
                                        if j[-1] != 'T':
                                            extracted_information['TICKER'].append(j)
                                            
                                            
df = pd.DataFrame(extracted_information)
print('DataFrame criado com sucesso')

DataFrame criado com sucesso


In [12]:
df.head(5)

,TICKER
0,BMGB4L
1,CEAC11
2,CEAC4L
3,BVEN3L
4,CEAC3L


# PARTE 3 | Yahoo Finance

Com o DataFrame de Tickers criado, iremos utilizar a biblioteca Yahoo Finance para extrair o Preço de Fechamento das ações no mercado à vista.

<pre>
Lembrete: Nós poderíamos ter extraído o Preço de Fechamento (e outras informações) diretamente do arquivo BVBG.028, durante a ETAPA 2 de manipulação do XML. Entretanto, ficariamos restritos a data em que o arquivo foi baixado (considerando que as informações disponíveis no BVBG são exclusivas da data do arquivo que baixamos)
</pre>

Ressalto que o DataFrame que criamos, acaba por ser uma lista TICKERs temporária, pois ele inclui o CODIGO DOS ATIVOS em diferentes mercados. Portanto, deveremos realizar um filtro específico (DataFrame final) para as informações que desejamos obter.

In [13]:
# Finanças
import pandas_datareader as web
import yfinance as yf

In [14]:
df['CONSULTA'] = df['TICKER'] + '.SA'
df.to_excel('lista_tickerB3_temp.xls')
df.head(10)

,TICKER,CONSULTA
0,BMGB4L,BMGB4L.SA
1,CEAC11,CEAC11.SA
2,CEAC4L,CEAC4L.SA
3,BVEN3L,BVEN3L.SA
4,CEAC3L,CEAC3L.SA
5,BVEN9L,BVEN9L.SA
6,ATWN11,ATWN11.SA
7,BVEN11,BVEN11.SA
8,FGPM11,FGPM11.SA
9,INNT13,INNT13.SA


In [15]:
# Exemplo de consulta utilizando o método get_data_yahoo
y_example = web.get_data_yahoo('PORD11.SA', start = "2022-02-08", end = "2022-02-08")
y_example

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2022-02-08,93.910004,92.519997,93.400002,93.910004,5003,93.910004


In [18]:
lista_yahoo = [i for i in df.CONSULTA]
print(lista_yahoo[0:30])

['BMGB4L.SA', 'CEAC11.SA', 'CEAC4L.SA', 'BVEN3L.SA', 'CEAC3L.SA', 'BVEN9L.SA', 'ATWN11.SA', 'BVEN11.SA', 'FGPM11.SA', 'INNT13.SA', 'PNVL4L.SA', 'PNVL3L.SA', 'SMFT3F.SA', 'RDOR3L.SA', 'LAME3L.SA', 'LAME4L.SA', 'SUZB3L.SA', 'DCOF31.SA', 'OC1F31.SA', 'DI1F31.SA', 'DDIF31.SA', 'SALI11.SA', 'HBRE3L.SA', 'BPAC3L.SA', 'BPAC9L.SA', 'ESPA3L.SA', 'INTB3L.SA', 'BPAC5L.SA', 'PLAS3F.SA', 'CCMU22.SA']


In [20]:
lista_yahoo = [i for i in df.CONSULTA]

final_data = {'TICKER_YAHOO':[],'PREÇO_FECHAMENTO':[]}

print(f'Favor tentar extrair um total de: {len(lista_yahoo)} ativos')
count = 0
for i in lista_yahoo[0:80]:    
    try:
        price = web.get_data_yahoo(i, start = "2022-02-08", end = "2022-02-08")
        tg_price = price['Close'].values[0]                         # Linha que estava errada
        final_data['TICKER_YAHOO'].append(i)
        final_data['PREÇO_FECHAMENTO'].append(tg_price)
        print(f'Extração {count} = Sucesso')
        count += 1
    except:
        pass

Favor tentar extrair um total de: 4021 ativos
Extração 0 = Sucesso
Extração 1 = Sucesso
Extração 2 = Sucesso
Extração 3 = Sucesso
Extração 4 = Sucesso


In [22]:
df_final = pd.DataFrame(final_data)
df_final.to_excel('arquivo_final.xls', index=False)
df_final

,TICKER_YAHOO,PREÇO_FECHAMENTO
0,BERK34.SA,84.279999
1,DISB34.SA,49.639999
2,AIRB34.SA,42.680000
3,PAGS34.SA,18.260000
4,BOVV11.SA,113.120003
